In [16]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import random

In [17]:
class multi_armed_bandit:
    def __init__(self, k, bandits):
        self.k = k
        self.bandits = bandits
        self.qn = np.zeros((self.bandits, self.k))
        self.true_rewards =  np.zeros((self.bandits, k))
        self.max_arms = np.argmax(self.true_rewards, 1)
        
    def random_walks(self):
        self.true_rewards += np.random.normal(0,0.01,(self.bandits, self.k))
        self.max_arms = np.argmax(self.true_rewards, 1)
        
    def increment_method(self, i, j, R, alpha):
        self.qn[i][j] += alpha*(R - self.qn[i][j])
        
    def sampled_average(self):
        pass

In [ ]:
fig1 = plt.figure().add_subplot(111)
fig2 = plt.figure().add_subplot(111)

for optimism in [0,5]:
    nonstationary_rewards = []
    nonstationary_opt_pulls = []
    nonstationary_testbed = multi_armed_bandit(10,2000)
    nonstationary_testbed.qn += optimism

    stationary_testbed = multi_armed_bandit(10,2000)
    stationary_rewards = []
    stationary_opt_pulls = []

    epsilon = 0

    for i in range(1001):
        stationary_opt_pull = 0
        nonstationary_opt_pull = 0
        stationary_temp_rewards = []
        nonstationary_temp_rewards = []
        for j in range(nonstationary_testbed.bandits):

            #non stationary
            if(random.random() < epsilon):
                p = np.random.randint(nonstationary_testbed.k)
            else:
                p = np.argmax(nonstationary_testbed.qn[j])

            if(p == testbed.max_arms[j]):
                nonstationary_opt_pull+=1

            curr_reward = np.random.normal(nonstationary_testbed.true_rewards[j][p], 1)
            temp_rewards.append(curr_reward)

            nonstationary_testbed.increment_method(j, p, curr_reward, 0.1)

            #stationary
            if(random.random() < epsilon):
                p = np.random.randint(stationary_testbed.k)
            else:
                p = np.argmax(stationary_testbed.qn[j])

            if(p == testbed.max_arms[j]):
                stationary_opt_pull+=1

            curr_reward = np.random.normal(stationary_testbed.true_rewards[j][p], 1)
            temp_rewards.append(curr_reward)

            stationary_testbed.increment_method(j, p, curr_reward, 0.1)

        nonstationary_testbed.random_walks()

        avg_reward = np.mean(temp_rewards)
        nonstationary_rewards.append(avg_reward)
        nonstationary_opt_pulls.append(float(nonstationary_opt_pull)/20)

        avg_reward = np.mean(temp_rewards)
        stationary_rewards.append(avg_reward)
        stationary_opt_pulls.append(float(stationary_opt_pull)/20)

    fig1.plot(range(1001), stationary_rewards, 'r')
    fig2.plot(range(1001), nonstationary_rewards, 'b')

    fig1.plot(range(1001), stationary_opt_pulls, 'r')
    fig2.plot(range(1001), nonstationary_opt_pulls, 'b')

fig1.title.set_text('Stationary Case')
fig2.title.set_text('Non-Stationary Case')